In [1]:
import os
import logging
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from sqlalchemy import create_engine

from neuralforecast import NeuralForecast # type: ignore
from neuralforecast.auto import AutoTimeMixer  # type: ignore
from neuralforecast.models import TimeMixer, SOFTS  # type: ignore
from neuralforecast.losses.pytorch import HuberLoss  # type: ignore

In [2]:
load_dotenv()  # take environment variables from .env.

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

alchemyEngine = create_engine(
    db_url,
    pool_recycle=3600,
)

In [3]:
# query = """
# with cte as (
# SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
# FROM index_daily_em_view
# where symbol = '399673'
# order by date desc
# limit 1200
# ) select * from cte order by date
# """
query = """
with cte as (
SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
FROM index_daily_em_view 
where symbol = '399673'
and change_rate is not null
) select * from cte order by date
"""

raw_df = pd.read_sql(query, alchemyEngine, parse_dates=["date"])

In [5]:
df = raw_df.rename(columns={"date":"ds", "change_rate":"y"})
df.insert(0, "unique_id", "399673")

In [6]:
df.dtypes

unique_id                     object
ds                    datetime64[ns]
open                         float64
close                        float64
high                         float64
low                          float64
volume                       float64
amount                       float64
open_preclose_rate           float64
high_preclose_rate           float64
low_preclose_rate            float64
vol_change_rate              float64
amt_change_rate              float64
y                            float64
dtype: object

In [7]:
len(df)

2488

In [8]:
df_train = df

## optional df_train

In [8]:
# handle univariate first, to avoid the following error
# 'DataFrame' object has no attribute 'temporal_cols'
df_train = df[["unique_id", "ds", "y", "open_preclose_rate"]]
df_train.tail()

,unique_id,ds,y,open_preclose_rate
2481,399673,2024-08-27,-0.87590,-0.56837
2482,399673,2024-08-28,-0.06461,-0.27391
2483,399673,2024-08-29,0.50911,-1.45864
2484,399673,2024-08-30,2.85494,0.22245
2485,399673,2024-09-02,-2.91054,-0.26187


In [7]:
# handle univariate first, to avoid the following error
# 'DataFrame' object has no attribute 'temporal_cols'
df_train = df[["unique_id", "ds", "y"]]

In [44]:
nf.models[0].metrics

{'train_loss': tensor(1.3861),
 'train_loss_step': tensor(1.3861),
 'train_loss_epoch': tensor(1.3861),
 'valid_loss': tensor(1.3116),
 'ptl/val_loss': tensor(1.3116)}

In [11]:
nf.models[0]

TimeMixer

In [45]:
metrics = nf.models[0].metrics

In [46]:
float(metrics["train_loss"])

1.386099934577942

In [47]:
float(metrics["valid_loss"])

1.3116085529327393

In [13]:
print(f"{nf.models[0].loss}  {nf.models[0].valid_loss}")

MAE()  MAE()


In [12]:
# list out all the attributes for `nf.models[0]`
attributes = dir(nf.models[0])
for attribute in attributes:
    print(attribute)

CHECKPOINT_HYPER_PARAMS_KEY
CHECKPOINT_HYPER_PARAMS_NAME
CHECKPOINT_HYPER_PARAMS_TYPE
EXOGENOUS_FUTR
EXOGENOUS_HIST
EXOGENOUS_STAT
SAMPLING_TYPE
T_destination
_LightningModule__check_allowed
_LightningModule__check_not_nested
_LightningModule__to_tensor
_TimeMixer__multi_scale_process_inputs
__annotations__
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattr__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__jit_unused_properties__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_apply
_apply_batch_transfer_handler
_automatic_optimization
_backward_hooks
_backward_pre_hooks
_buffers
_call_batch_hook
_call_impl
_check_exog
_compiled_call_impl
_compiler_ctx
_create_windows
_current_fx_name
_device
_device_mesh
_dtype
_example_input_array
_fabric
_fabric_optimizers
_fit
_fit_distributed
_forward_hooks
_forward_hooks_always_call

In [48]:
train_losses = nf.models[0].train_trajectories
train_losses

[(0, 1.2652792930603027),
 (1, 1.4556605815887451),
 (2, 1.913306474685669),
 (3, 1.648797631263733),
 (4, 1.6551233530044556),
 (5, 1.3696376085281372),
 (6, 1.5856698751449585),
 (7, 1.5500593185424805),
 (8, 1.4477684497833252),
 (9, 1.6019489765167236),
 (10, 1.5306743383407593),
 (11, 1.6930204629898071),
 (12, 1.2258554697036743),
 (13, 1.426756739616394),
 (14, 1.477355718612671),
 (15, 1.4579471349716187),
 (16, 1.6078250408172607),
 (17, 1.4272948503494263),
 (18, 1.4014198780059814),
 (19, 1.6109946966171265),
 (20, 1.7012355327606201),
 (21, 1.4736907482147217),
 (22, 1.4918445348739624),
 (23, 1.5586013793945312),
 (24, 1.4386353492736816),
 (25, 1.4953359365463257),
 (26, 1.3780930042266846),
 (27, 1.5182558298110962),
 (28, 1.455914855003357),
 (29, 1.5370842218399048),
 (30, 1.5447022914886475),
 (31, 1.468395471572876),
 (32, 1.3750426769256592),
 (33, 1.3290830850601196),
 (34, 1.4182649850845337),
 (35, 1.4127308130264282),
 (36, 1.5263346433639526),
 (37, 1.618616819

In [49]:
valid_losses = nf.models[0].valid_trajectories
valid_losses

[(0, 1.2441478967666626),
 (100, 1.2389733791351318),
 (200, 1.2499414682388306),
 (300, 1.2595993280410767),
 (400, 1.2697396278381348),
 (500, 1.314969778060913),
 (600, 1.3116085529327393)]

In [50]:
# get the smallest loss from `train_losses`, which is a list of 2-element tuples, and the 2nd element is loss value.
smallest_loss = min(train_losses, key=lambda x: x[1])[1]
smallest_loss

1.1313724517822266

In [51]:
min(valid_losses, key=lambda x: x[1])[1]


1.2389733791351318

# Troubleshoot

## RuntimeError('The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 2')

In [9]:
hparams_raw = {
    "h": 20,
    "d_ff": 32,
    "top_k": 9,
    "d_model": 32,
    "dropout": 0.025678663612217545,
    "e_layers": 15,
    "use_norm": False,
    "optimizer": "SGD",
    "batch_size": 64,
    "covar_dist": [
        0.012446164040523651,
        0.07434833727501454,
        0.005954535986635773,
        0.0459394531992627,
        0.1216363214684387,
        0.31218214537049555,
        0.008661552616809226,
        0.0914576620330267,
        0.10292342609136783,
        0.01150455517290314,
        0.006043906561017433,
        0.04094889296331747,
        0.08992125574660075,
        0.07603179147458679,
    ],
    "input_size": 678,
    "moving_avg": 25,
    "topk_covar": 7,
    "decomp_method": "moving_avg",
    "learning_rate": 0.00020395846998135436,
    "local_scaler_type": "robust-iqr",
    "channel_independence": 1,
    "down_sampling_layers": 2,
    "down_sampling_method": "conv",
    "down_sampling_window": 13,
    "early_stop_patience_steps": 15,
    "accelerator": "cpu",
    "devices": 1,
    "max_steps": 1000,
    "val_check_steps": 50,
    "validate": True,
}
# hparams_raw = {
#     "batch_size": 32,
#     "channel_independence": 0,
#     "covar_dist": [
#         0.05941820259601838,
#         0.004731065485515826,
#         0.14343522095659036,
#         0.09697409120321702,
#         0.10287015087717696,
#         0.06997545077365976,
#         0.022408840619061114,
#         0.018386297819442635,
#         0.0664151536605972,
#         0.02222760702564122,
#         0.07518915528342811,
#         0.0804237066160445,
#         0.04750083522628452,
#         0.19004422185732242,
#     ],
#     "d_ff": 128,
#     "d_model": 256,
#     "decomp_method": "moving_avg",
#     "down_sampling_layers": 7,
#     "down_sampling_method": "max",
#     "down_sampling_window": 4,
#     "dropout": 0.33971283243512906,
#     "e_layers": 15,
#     "early_stop_patience_steps": 11,
#     "input_size": 13,
#     "learning_rate": 0.001032553544411743,
#     "local_scaler_type": "standard",
#     "moving_avg": 33,
#     "optimizer": "Adam",
#     "top_k": 6,
#     "use_norm": True,
#     "h": 20,
#     "max_steps": 1000,
#     "validate": True,
#     "random_seed": 7,
#     "accelerator": "cpu",
#     "devices": 1,
# }

In [10]:
# remove "validate", "covar_dist" from hparams_raw dict
validate = hparams_raw.pop('validate', None)
hparams_raw.pop('covar_dist', None)
hparams_raw.pop("topk_covar", None)
local_scaler_type = hparams_raw.pop("local_scaler_type")

In [11]:
import torch
import torch.optim as optim
from torch.optim.optimizer import Optimizer

def _select_optimizer(**kwargs):
    match kwargs["optimizer"]:
        case "Adam":
            model_optim = optim.Adam
        case "AdamW":
            model_optim = optim.AdamW
        case "SGD":
            model_optim = optim.SGD
    optim_args = {
        "lr": kwargs["learning_rate"],
    }
    if kwargs["optimizer"] != "SGD":
        optim_args["fused"] = (
            kwargs["accelerator"] in ("gpu", "auto") and torch.cuda.is_available()
        )
    return model_optim, optim_args

In [12]:
optimizer, optim_args = _select_optimizer(**hparams_raw)
hparams_raw.pop("optimizer")

'SGD'

In [13]:
seed_logger = logging.getLogger("lightning_fabric.utilities.seed")
orig_seed_log_level = seed_logger.getEffectiveLevel()
seed_logger.setLevel(logging.FATAL)

tmx = TimeMixer(
    n_series=1,
    loss=HuberLoss(),
    enable_progress_bar=False,
    enable_model_summary=False,
    optimizer=optimizer,
    optimizer_kwargs=optim_args,
    **hparams_raw,
)

nf = NeuralForecast(
    models=[tmx],
    freq="B",
    local_scaler_type=local_scaler_type,
)

seed_logger.setLevel(orig_seed_log_level)

In [14]:
# validate = True
val_size = min(300, int(len(df_train) * 0.9)) if validate else 0
nf.fit(df_train, val_size=val_size)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/jx/ProgramData/git/neuralforecast/neuralforecast/common/_base_model.py:379: UserWarning: ignoring learning rate passed in optimizer_kwargs, using the model's learning rate
  warnings.warn(
/Users/jx/.pyenv/versions/3.12.2/envs/venv_3.12.2/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [13]:
import math
math.ceil(0.00000001)

1

In [20]:
import torch
import torch.nn as nn
import math


class MovingAvg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """

    def __init__(self, kernel_size, stride):
        super(MovingAvg, self).__init__()
        if kernel_size <= 1:
            raise ValueError("kernel_size must be greater than 1")
        self.kernel_size = kernel_size
        self.stride = stride
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        padding = (self.kernel_size - 1) // 2
        front = x[:, 0:1, :].repeat(1, padding, 1)
        end = x[:, -1:, :].repeat(1, padding + (self.kernel_size % 2 == 0), 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


# Example usage
x = torch.randn(1, 57, 10)  # Example input tensor
kernel_size = 20
stride = 1
model = MovingAvg(kernel_size, stride)
output = model(x)
print(output.shape)  # Should print torch.Size([1, 56, 10])

torch.Size([1, 57, 10])
